# Predictions blender

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import shutil

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [12]:
import numpy as np
import pandas as pd

from data_utils import get_id_type_list_for_class, GENERATED_DATA, OUTPUT_PATH
from test_utils import create_submission

## Simple probability mean 

In [6]:
predictions_csv = glob(os.path.join(OUTPUT_PATH, "predictions_*.csv"))
predictions_csv

['D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_0_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_10_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_11_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_12_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_13_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_14_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_1_SqueezeNet21_all_classes_fold=4_seed=

In [7]:
predictions = []
for csv_filepath in predictions_csv:
    predictions.append(pd.read_csv(csv_filepath))

In [18]:
predictions[0].sort_values(by='image_name').head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
12574,file_0,0.066559,0.026386,0.022647,0.027997,0.013708,0.943612,0.019972,0.032191,0.064726,0.007336,0.038537,0.124131,0.932207,0.030553,0.022448,0.060615,0.021036
55285,file_1,0.038082,0.014214,0.010752,0.015895,0.007087,0.972440,0.007770,0.019515,0.043018,0.002502,0.017839,0.100316,0.956652,0.018208,0.010303,0.034832,0.010925
8330,file_10,0.531736,0.088515,0.147094,0.066210,0.053297,0.578340,0.081342,0.103375,0.219836,0.139833,0.157908,0.314950,0.895389,0.175684,0.085034,0.094970,0.237179
19435,file_100,0.180771,0.013756,0.018632,0.009134,0.013630,0.951660,0.010162,0.021247,0.061058,0.009143,0.032453,0.117525,0.971390,0.055973,0.015998,0.061598,0.047556
54655,file_1000,0.244277,0.051670,0.078124,0.059738,0.039378,0.754265,0.064522,0.097159,0.168164,0.050973,0.091554,0.297560,0.866799,0.116241,0.045633,0.082810,0.119394


In [19]:
predictions[1].sort_values(by='image_name').head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
16525,file_0,0.073166,0.029202,0.025499,0.028145,0.016391,0.949343,0.018040,0.036076,0.075053,0.007552,0.037240,0.133683,0.930354,0.032100,0.023555,0.064420,0.022039
60335,file_1,0.030922,0.017255,0.010863,0.015595,0.008960,0.966943,0.008244,0.024155,0.054117,0.002726,0.022738,0.113144,0.956747,0.015055,0.012653,0.040333,0.009813
38579,file_10,0.313379,0.051428,0.100774,0.047588,0.058047,0.721346,0.031028,0.072293,0.221484,0.057846,0.107129,0.291878,0.893824,0.121176,0.054719,0.104063,0.139087
21391,file_100,0.151895,0.045386,0.058039,0.066351,0.020819,0.876061,0.031993,0.077704,0.123705,0.029359,0.056405,0.235682,0.886710,0.076777,0.047129,0.048100,0.066734
24228,file_1000,0.479737,0.062571,0.088518,0.045429,0.046464,0.707323,0.058361,0.065683,0.153167,0.063890,0.158736,0.298752,0.882938,0.228939,0.037973,0.101017,0.173090


In [8]:
def compute_mean(predictions):
    df = predictions[0]
    for p in predictions[1:]:
        df = pd.concat([df, p], axis=0)
    df = df.apply(pd.to_numeric, errors='ignore')
    gb = df.groupby('image_name')
    df2 = gb.agg(np.mean).reset_index()
    return df2

In [9]:
mean_predictions_df = compute_mean(predictions)

In [10]:
mean_predictions_df.head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,file_0,0.075577,0.022042,0.028876,0.018089,0.032277,0.903471,0.013046,0.059221,0.093378,0.027963,0.020402,0.083580,0.967447,0.053680,0.035005,0.046575,0.088839
1,file_1,0.035373,0.011506,0.014149,0.007825,0.020209,0.938606,0.004963,0.037624,0.064372,0.013600,0.006733,0.056471,0.984725,0.029025,0.024612,0.022560,0.068315
2,file_10,0.449505,0.034339,0.059886,0.027366,0.057424,0.431175,0.063071,0.088553,0.169601,0.047239,0.087217,0.583021,0.913226,0.166780,0.118563,0.112523,0.232856
3,file_100,0.078833,0.020902,0.019649,0.012992,0.025159,0.939492,0.007644,0.047085,0.070833,0.030681,0.010401,0.059309,0.973639,0.079619,0.026858,0.032322,0.110833
4,file_1000,0.281819,0.043391,0.059090,0.035357,0.050177,0.832329,0.022377,0.087638,0.141859,0.083981,0.046980,0.133932,0.937326,0.180613,0.047837,0.055389,0.167381


In [11]:
thresholds = {
    'agriculture': 0.35,
    'artisinal_mine': 0.35,
    'bare_ground': 0.35,
    'blooming': 0.35,
    'blow_down': 0.1,
    'clear': 0.5,
    'cloudy': 0.35,
    'conventional_mine': 0.35,
    'cultivation': 0.35,
    'habitation': 0.35,
    'haze': 0.35,
    'partly_cloudy': 0.35,
    'primary': 0.5,
    'road': 0.35,
    'selective_logging': 0.35,
    'slash_burn': 0.1,
    'water': 0.35
}

In [14]:
create_submission(mean_predictions_df, info="squeezenet21_mean_5_folds", thresholds=thresholds)

## Train xgboost trees to make better predictions

In [33]:
val_predictions_csv = glob(os.path.join(OUTPUT_PATH, "val_predictions_*.csv"))
val_predictions_csv

['D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\val_predictions_SqueezeNet21_all_classes_fold=0_seed=2017_run=1_2017-07-10-13-19.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\val_predictions_SqueezeNet21_all_classes_fold=0_seed=2017_run=2_2017-07-10-13-19.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\val_predictions_SqueezeNet21_all_classes_fold=1_seed=2017_run=1_2017-07-10-13-19.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\val_predictions_SqueezeNet21_all_classes_fold=1_seed=2017_run=2_2017-07-10-13-19.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\val_predictions_SqueezeNet21_all_classes_fold=2_seed=2017_run=1_2017-07-10-13-19.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\val_predictions_SqueezeNet21_all_classes_fold=2_seed=2017_run=2_2017-07-10-13-19.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\va

In [73]:
df = pd.read_csv(val_predictions_csv[0]).dropna()
for filepath in val_predictions_csv[1:]:
    df = pd.concat([df, pd.read_csv(filepath).dropna()])
df.reset_index(inplace=True)    

In [83]:
print(len(df))
df.head()

80640


,index,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,...,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water,image_id,target
0,0,test_628,0.071995,0.019810,0.018135,0.020460,0.015155,0.950393,0.018868,0.034112,...,0.007275,0.032921,0.148809,0.933757,0.041442,0.018702,0.055927,0.026246,628,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,1,test_8842,0.070470,0.024685,0.020838,0.020732,0.016295,0.946937,0.013844,0.024798,...,0.005569,0.047688,0.121639,0.943046,0.033911,0.019459,0.070846,0.021167,8842,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2,test_11877,0.760127,0.092261,0.118277,0.099680,0.101711,0.741155,0.107806,0.115136,...,0.298347,0.111120,0.299565,0.822521,0.437350,0.100299,0.150043,0.422677,11877,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, ..."
3,3,test_2413,0.229211,0.058638,0.096851,0.112940,0.040866,0.833260,0.051784,0.093998,...,0.053805,0.081282,0.317577,0.914311,0.087564,0.074766,0.100207,0.135809,2413,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,4,test_28838,0.071773,0.017712,0.019611,0.027369,0.016949,0.962932,0.013631,0.027329,...,0.006570,0.035925,0.116127,0.937008,0.026963,0.020253,0.056307,0.017759,28838,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [107]:
df['image_id'] = df['image_name'].apply(lambda x: int(x[len('test_'):]))

In [77]:
from data_utils import unique_tags, get_label

In [80]:
get_label(df.loc[0, 'image_id'], "Train_jpg")

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=uint8)

In [81]:
df['target'] = df['image_id'].apply(lambda image_id: get_label(image_id, "Train_jpg"))

In [84]:
df.loc[0,['image_id', 'target']]

image_id                                                  628
target      [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...
Name: 0, dtype: object

#### Check prediction of rare classes

In [92]:
from data_utils import equalized_data_classes, TRAIN_ENC_CL_CSV

In [97]:
image_ids = TRAIN_ENC_CL_CSV[TRAIN_ENC_CL_CSV['class_0'] == 1]['image_id']

In [102]:
TRAIN_ENC_CL_CSV[TRAIN_ENC_CL_CSV['image_id'] == 10]

,image_name,tags,cloudy,selective_logging,cultivation,artisinal_mine,water,bare_ground,road,primary,...,habitation,blooming,haze,agriculture,class_0,class_1,class_2,class_3,class_4,image_id
10,train_10,agriculture clear primary slash_burn water,0,0,0,0,1,0,0,1,...,0,0,0,1,1,0,1,1,1,10


In [110]:
rare_classes_predictions = df[df['image_id'].isin(image_ids)]

In [116]:
rare_classes_predictions.loc[17, list(tuple(unique_tags) + ('target', ))]

agriculture                                                  0.0397602
artisinal_mine                                               0.0172498
bare_ground                                                  0.0159017
blooming                                                     0.0199326
blow_down                                                    0.0100598
clear                                                         0.973735
cloudy                                                         0.00855
conventional_mine                                            0.0235309
cultivation                                                  0.0542153
habitation                                                  0.00319715
haze                                                         0.0224533
partly_cloudy                                                  0.10198
primary                                                        0.95575
road                                                         0.0193523
select

In [85]:
import xgboost as xgb

d:\vfomin\myexamples\venv3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### A simple try of xgb

In [118]:
from sklearn.model_selection import KFold

In [119]:
n_folds = 5

In [141]:
kf = KFold(n_splits=n_folds)

trainval_x = df[unique_tags].values
trainval_y = np.zeros_like(trainval_x)
for i in range(len(trainval_y)):
    trainval_y[i, :] = df['target'].values[i]
    
for train_index, test_index in kf.split(trainval_x):
    train_x, val_x = trainval_x[train_index], trainval_x[test_index]
    train_y, val_y = trainval_y[train_index], trainval_y[test_index]
    
    print(train_x.shape, train_y.shape)
    print(train_x[:5, :], train_y[:5])
    print(val_x[:5, :], val_y[:5])
    break

(64512, 17) (64512, 17)
[[ 0.09093951  0.02054392  0.02418912  0.01544789  0.03247925  0.92379266
   0.00828369  0.04989937  0.10721629  0.0278988   0.00523409  0.08840037
   0.97719461  0.06283034  0.03732468  0.03562859  0.09488261]
 [ 0.05455842  0.01311656  0.02213255  0.01007489  0.03110504  0.91531706
   0.00602263  0.0469023   0.12840895  0.02287883  0.00794587  0.05563896
   0.98842609  0.03919864  0.03108688  0.02699875  0.06988449]
 [ 0.01405455  0.00637132  0.00833274  0.00247492  0.01473563  0.95992196
   0.00172729  0.03102464  0.03605482  0.00775872  0.00121341  0.03003966
   0.99548018  0.01660914  0.023029    0.01427789  0.05433382]
 [ 0.45359641  0.01025823  0.04004937  0.00729494  0.03303159  0.48314595
   0.02731058  0.06863047  0.13719328  0.02727759  0.02906365  0.58874065
   0.92141306  0.10537007  0.12347737  0.07908679  0.10379606]
 [ 0.1899071   0.01707581  0.02652191  0.01242715  0.06425259  0.72012264
   0.03139025  0.06736487  0.09192603  0.04307153  0.02066

In [192]:
OneVsRestClassifier??

In [158]:
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier

In [188]:
params = {
    "objective": "binary:logistic",
    # "num_class": len(unique_tags),
    # "booster": "gbtree",
    # "eval_metric": "mlogloss",
    "learning_rate": 0.01,
    # "tree_method": 'exact',
    "n_estimators": 100,
    "max_depth": 6,
    # "subsample": subsample,
    # "colsample_bytree": colsample_bytree,
    "silent": False,
    # "seed": random_state,
}

In [189]:
target_clf = OneVsRestClassifier(XGBClassifier(**params))

In [190]:
target_clf.fit(train_x[:100,:], train_y[:100,:])

OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=1, subsample=1),
          n_jobs=1)

In [191]:
target_clf.get_params(deep=True)

{'estimator': XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
        gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=6,
        min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
        objective='binary:logistic', reg_alpha=0, reg_lambda=1,
        scale_pos_weight=1, seed=0, silent=1, subsample=1),
 'estimator__base_score': 0.5,
 'estimator__colsample_bylevel': 1,
 'estimator__colsample_bytree': 1,
 'estimator__gamma': 0,
 'estimator__learning_rate': 0.01,
 'estimator__max_delta_step': 0,
 'estimator__max_depth': 6,
 'estimator__min_child_weight': 1,
 'estimator__missing': None,
 'estimator__n_estimators': 100,
 'estimator__nthread': -1,
 'estimator__objective': 'binary:logistic',
 'estimator__reg_alpha': 0,
 'estimator__reg_lambda': 1,
 'estimator__scale_pos_weight': 1,
 'estimator__seed': 0,
 'estimator__silent': 1,
 'estimator__subsample': 1,
 'n_jobs': 1}